In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from plotly.subplots import make_subplots
import plotly.graph_objects as go

#BY Xiaotian Zhu

In [2]:
def get_stock_history_from_yahoo(ticker) :
    
    #
    # first build the url string from the ticker
    #
    urlstr = "https://finance.yahoo.com/quote/{0}/history?p={0}".format(ticker)

    # 
    # go to the html page to fetch the price history table
    # trim the table to get what we really want
    # 
    hdata = pd.read_html(urlstr)[0][:-1]
    
    #remove the dividend line
    hdata=hdata.drop_duplicates(subset='Date',inplace=False)
    #
    # use the 'Date' colume as the index
    #
    #hdata = hdata.set_index('Date')
    
    #
    # The 'Date' is not parse as DateTime, so let's convert them
    #
    #hdata.index = pd.to_datetime(hdata.index)

    # 
    # the rest of the data come in as 'object' type 
    # let's turn them into the appropriate data type 
    #
    #hdata = hdata.astype('d')
    
    #
    # Let's beautiful the data column names
    #
    hdata.columns = ['Date','Open','High','Low','Close', 'AdjClose','Volume']

    #
    # all set
    #
    return hdata

In [3]:
def window(l,window_size):
    moving_average=[]
    i=0
    while i<len(l)-window_size+1:
        thisWindow=l[i:i+window_size]
        window_average=sum(thisWindow)/window_size
        moving_average.append(window_average)
        i+=1
    return moving_average
    

In [4]:
def get_movingAve(ticker,length):
    df=get_stock_history_from_yahoo(ticker)
    df1=df[[ 'Date','AdjClose']].copy()
    l1=df1['AdjClose'].astype(float).values.tolist()
    return window(l1,length)


In [56]:
import os
if not os.path.exists("images"):
    os.mkdir("images")

In [68]:
def  plot_candlestick(ticker,short=5,long=15):
    
    orig_data=get_stock_history_from_yahoo(ticker)
    trace1= go.Scatter(x=orig_data['Date'],y=get_movingAve(ticker,short),mode = "lines",name = 'short moving average window ')
    trace2=go.Scatter(x=orig_data['Date'],y=get_movingAve(ticker,long),mode = "lines",name = 'long moving average window ')
    trace3=go.Candlestick(x=orig_data['Date'],
                open=orig_data['Open'],
                high=orig_data['High'],
                low=orig_data['Low'],
                close=orig_data['Close'],name='Candlestick')
    trace4=go.Scatter(x=orig_data['Date'], y=orig_data['Volume'],name='Volume')
    fig = make_subplots(rows=2, cols=1)
    fig.add_trace(
        trace1,row=1, col=1
    )
    fig.add_trace(
        trace2,row=1, col=1
    )
    fig.add_trace(
        trace3,row=1, col=1
    )
    fig.add_trace(
        trace4,row=2, col=1
    )
    
    fig.update_layout(xaxis_rangeslider_visible=False)
    fig.write_image("images/fig1.png")
    
    return fig.show()

In [69]:
plot_candlestick('AMZN',short=5,long=15)

In [70]:
plot_candlestick('GE',5,10)


In [71]:
plot_candlestick('FDX')